In [1]:
import pandas as pd
import numpy as npcategory_encoders
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import requests
from mosaicml import *category_encoders
from mosaicml.constants import MLModelFlavours

ModuleNotFoundError: No module named 'category_encoders'

In [4]:
!pip install category_encoders
# from mosaicml.constants import MLModelFlavours

     |████████████████████████████████| 81kB 924kB/s eta 0:00:011
     |████████████████████████████████| 12.2MB 2.8MB/s eta 0:00:01
     |████████████████████████████████| 18.3MB 51.4MB/s eta 0:00:01
     |████████████████████████████████| 13.0MB 54.1MB/s eta 0:00:01
     |████████████████████████████████| 235kB 50.7MB/s eta 0:00:01
     |████████████████████████████████| 38.5MB 49.3MB/s eta 0:00:01
     |████████████████████████████████| 10.8MB 50.7MB/s eta 0:00:01
     |████████████████████████████████| 307kB 33.8MB/s eta 0:00:01
     |████████████████████████████████| 348kB 59.8MB/s eta 0:00:01
     |████████████████████████████████| 512kB 58.7MB/s eta 0:00:01
     |████████████████████████████████| 235kB 47.0MB/s eta 0:00:01
     |████████████████████████████████| 61kB 9.1MB/s  eta 0:00:01
ERROR: refractio 2.1.5.2 has requirement pandas==2.0.0, but you'll have pandas 2.2.1 which is incompatible.
ERROR: mlflow 2.6.0 has requirement pytz<2024, but you'll have pytz 2024.1 which is in

In [2]:
cars = pd.read_csv("/data/car_new.csv")
cars

,buying_num,maintenance_num,doors_num,persons_num,lugboot_num,safety_num,class_num
0,3,3,0,0,2,2,2
1,3,3,0,0,2,0,2
2,3,3,0,0,1,1,2
3,3,3,0,0,1,2,2
4,3,3,0,0,1,0,2
...,...,...,...,...,...,...,...
1722,1,1,3,2,1,2,1
1723,1,1,3,2,1,0,3
1724,1,1,3,2,0,1,2
1725,1,1,3,2,0,2,1


In [3]:
Y = cars['class_num']
X = cars.drop (['class_num'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.18, shuffle=True, random_state=25)
print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(1416, 6) (311, 6) (1416,) (311,)


In [4]:
first_tree = DecisionTreeClassifier()
model=first_tree.fit(X_train, y_train)
model

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [17]:
y_pred = model.predict(X_test)
first_tree.score(X_test, y_test)
y_prob = model.predict_proba(X_test)[:,1]

In [6]:
@scoring_func
def score(model, request):
    payload = request.json["payload"]
    data = pd.DataFrame(eval(payload))
    prediction = pd.Series(model.predict(data))
    return prediction.tolist()

In [7]:
req = requests.Request()
req.json = {"payload":X_train.head(1).to_json()}

sch = generate_schema(score,(model, req), X_train)
print(sch)

{'input': {'type': 'pd.core.frame.DataFrame', 'items': {'type': 'object', 'properties': {'buying_num': {'type': 'integer', 'format': 'int64'}, 'maintenance_num': {'type': 'integer', 'format': 'int64'}, 'doors_num': {'type': 'integer', 'format': 'int64'}, 'persons_num': {'type': 'integer', 'format': 'int64'}, 'lugboot_num': {'type': 'integer', 'format': 'int64'}, 'safety_num': {'type': 'integer', 'format': 'int64'}}}, 'example': [{'buying_num': 1, 'maintenance_num': 2, 'doors_num': 0, 'persons_num': 2, 'lugboot_num': 0, 'safety_num': 1}, {'buying_num': 3, 'maintenance_num': 2, 'doors_num': 2, 'persons_num': 2, 'lugboot_num': 2, 'safety_num': 2}]}, 'output': {'type': 'array', 'items': {'type': 'object'}, 'example': [2]}}


In [46]:
register_model(model, 
               score, 
               name='car_evaluation', 
               description='car evaluation classification model using sklearn', 
               flavour=MLModelFlavours.sklearn,
               schema=sch,
               y_true=y_test, 
               y_pred=y_pred,
               y_train=y_train, 
               y_test=y_test,
               x_train=X_train, 
               x_test=X_test, 
               prob=y_prob,
               labels=[0,1], 
               init_script="",
               model_type="classification",
               input_type="json", 
               explain_ai=True, 
               feature_names=list(X_train), 
               feature_ids=list(X_train),
               target_names=[0,1,2,3], 
               kyd=True, 
               kyd_score = False
              )

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


Publishing:   0%|          | 0/1 [00:00<?, ?it/s]

Know Your Data for 
/54d3a627-13ba-4ea8-b4e6-4c8f9d02bbc6/54d3a627-13ba-4ea8-b4e6-4c8f9d02bbc6/ Published

Validation                                                                        COMPLETED ✔
Data Initialised                                                                  COMPLETED ✔
Data Type Inferencing                                                             COMPLETED ✔
Data Drift Profile Insights                                                       COMPLETED ✔
Data Drift Stats Computation                                                      COMPLETED ✔
KYD Triggered Flag Set                                                            COMPLETED ✔

2022-05-27 13:28:57,833 - skater.core.explanations - WARNING - Classifiers with probability scores can be explained 
more granularly than those without scores. If a prediction method with 
scores is available, use that instead.
  X = X[:, np.newaxis]

2022-05-27 13:28:57,975 - skater.core.explanations - WARNING - Progress bars slow down runs by 10-20%. For slightly 
faster runs, do progressbar=False
2022-05-27 13:28:57,977 - skater.core.explanations - WARNING - Multiprocessing failed, going single process


...

  X = X[:, np.newaxis]

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float

  X = X[:, np.newaxis]



[5/6] features ████████████████---- Time elapsed: 0 seconds

  X = X[:, np.newaxis]

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float

  X = X[:, np.newaxis]



[6/6] features ████████████████████ Time elapsed: 0 seconds

2022-05-27 13:29:23,047 - skater.core.global_interpretation.tree_surrogate - INFO - post pruning applied ...
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)

2022-05-27 13:29:23,057 - skater.core.global_interpretation.tree_surrogate - INFO - Scorer used f1-score
2022-05-27 13:29:23,060 - skater.core.global_interpretation.tree_surrogate - INFO - post pruning applied ...
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_encoded = np.zeros(y.shape, dtype=np.int)

2022-05-27 13:29:23,067 - skater.core.global_interpretation.tree_surrogate - INFO - Scorer used f1-score
2022-05-27 13:29:23,070 - skater.core.global_interpretation.tree_surrogate - INFO - original score using base model 1.0
  'precision', 'predicted', average, warn_for)

  'precision', 'predicted', average, warn_for)

2022-05-27 13:29:2